In [1]:
import pandas as pd

### Input files

In [2]:
p15 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Final Blueprint runs\Final Blueprint (s24)\BAUS v2.25 - FINAL VERSION\run182_parcel_data_2015.csv',
                  usecols = ['parcel_id','fbpchcat','residential_units', 'deed_restricted_units', 'preserved_units', 'inclusionary_units', 'subsidized_units'])
p15.columns = list(p15)[:2] + [x+'_15' for x in list(p15)[2:]]

p50 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Final Blueprint runs\Final Blueprint (s24)\BAUS v2.25 - FINAL VERSION\run182_parcel_data_2050_UBI.csv',
                  usecols = ['parcel_id','fbpchcat','residential_units', 'deed_restricted_units', 'preserved_units', 'inclusionary_units', 'subsidized_units'])
p50.columns = list(p50)[:2] + [x+'_50' for x in list(p50)[2:]]
print(list(p50))

df = p15.merge(p50, on=['parcel_id', 'fbpchcat'], how='outer')

['parcel_id', 'fbpchcat', 'residential_units_50', 'deed_restricted_units_50', 'preserved_units_50', 'inclusionary_units_50', 'subsidized_units_50']


In [3]:
pg = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Current PBA50 Large General Input Data\2021_01_12_parcels_geography.csv',
                 usecols = ['PARCEL_ID', 'fbp_tra_id', 'fbp_sesit_id'])
print(pg.shape)

df = df.merge(pg, left_on='parcel_id', right_on='PARCEL_ID', how='left')

p_coc = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_cocid.csv')
p_coc['PARCEL_ID'] = p_coc['PARCEL_ID'].apply(lambda x: int(round(x)))
df = df.merge(p_coc, on='PARCEL_ID', how='left')

for col in ['preserved_units_50', 'preserved_units_15', 'subsidized_units_50', 'subsidized_units_15']:
    df[col] = df[col].fillna(0)

df['preserved_units_chg'] = df['preserved_units_50'] - df['preserved_units_15']
df['subsidized_units_chg'] = df['subsidized_units_50'] - df['subsidized_units_15']

(1956208, 3)


In [4]:
print(df.fbp_tra_id.unique())
print(df.fbp_sesit_id.unique())
print(df.coc_id.unique())

[nan 'tra3' 'tra2c' 'tra2a' 'tra1' 'tra2b']
['HRADIS' 'DIS' nan 'HRA']
[nan 'CoC']


### TRAs and HRAs

In [5]:
df_trahra = df.loc[(df.fbp_tra_id.notnull()) & (
                   (df.fbp_sesit_id == 'HRADIS') | (df.fbp_sesit_id == 'HRA'))][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_trahra.columns = ['metrics', 'value']
df_trahra['category'] = 'TRAs/HRAs'
df_trahra

,metrics,value,category
0,preserved_units_chg,151378.0,TRAs/HRAs
1,subsidized_units_chg,53449.0,TRAs/HRAs


In [6]:
df_tranonhra = df.loc[(df.fbp_tra_id.notnull()) & (
                      (df.fbp_sesit_id == 'DIS') | (df.fbp_sesit_id.isnull()))][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_tranonhra.columns = ['metrics', 'value']
df_tranonhra['category'] = 'TRAs/nonHRAs'
df_tranonhra

,metrics,value,category
0,preserved_units_chg,267417.0,TRAs/nonHRAs
1,subsidized_units_chg,61789.0,TRAs/nonHRAs


In [7]:
df_nontrahra = df.loc[(df.fbp_tra_id.isnull()) & (
                       (df.fbp_sesit_id == 'HRADIS') | (df.fbp_sesit_id == 'HRA'))][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_nontrahra.columns = ['metrics', 'value']
df_nontrahra['category'] = 'nonTRAs/HRAs'
df_nontrahra

,metrics,value,category
0,preserved_units_chg,10902.0,nonTRAs/HRAs
1,subsidized_units_chg,3432.0,nonTRAs/HRAs


In [8]:
df_other = df.loc[(df.fbp_tra_id.isnull()) & (
                  (df.fbp_sesit_id == 'DIS') | (df.fbp_sesit_id.isnull()))][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_other.columns = ['metrics', 'value']
df_other['category'] = 'other'
df_other

,metrics,value,category
0,preserved_units_chg,6365.0,other
1,subsidized_units_chg,1858.0,other


In [9]:
df_total = df[['preserved_units_chg','subsidized_units_chg']].sum().reset_index()
df_total.columns = ['metrics', 'value']
df_total['category'] = 'total'
df_total

,metrics,value,category
0,preserved_units_chg,436062.0,total
1,subsidized_units_chg,120528.0,total


### CoCs and HRAs

In [10]:
df_coc = df.loc[df.coc_id == 'CoC'][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_coc.columns = ['metrics', 'value']
df_coc['category'] = 'CoCs'
df_coc

,metrics,value,category
0,preserved_units_chg,118830.0,CoCs
1,subsidized_units_chg,39302.0,CoCs


In [11]:
df_hra = df.loc[(df.fbp_sesit_id == 'HRADIS') | (df.fbp_sesit_id == 'HRA')][[
    'preserved_units_chg','subsidized_units_chg']].sum().reset_index()

df_hra.columns = ['metrics', 'value']
df_hra['category'] = 'HRAs'
df_hra

,metrics,value,category
0,preserved_units_chg,162280.0,HRAs
1,subsidized_units_chg,56881.0,HRAs


### Combine and export

In [12]:
df_summary = pd.DataFrame(columns = ['metrics', 'value', 'category'])

for i in [df_trahra, df_tranonhra, df_nontrahra, df_other, df_total]:
    df_summary = pd.concat([df_summary, i], axis=0)
    
df_summary.sort_values(by=['metrics','category'],inplace=True)
display(df_summary)
df_summary.to_csv('final_blueprint_DR_types_TRAs.csv', index=False)

,metrics,value,category
0,preserved_units_chg,151378.0,TRAs/HRAs
0,preserved_units_chg,267417.0,TRAs/nonHRAs
0,preserved_units_chg,10902.0,nonTRAs/HRAs
0,preserved_units_chg,6365.0,other
0,preserved_units_chg,436062.0,total
1,subsidized_units_chg,53449.0,TRAs/HRAs
1,subsidized_units_chg,61789.0,TRAs/nonHRAs
1,subsidized_units_chg,3432.0,nonTRAs/HRAs
1,subsidized_units_chg,1858.0,other
1,subsidized_units_chg,120528.0,total


In [13]:
df_summary = pd.DataFrame(columns = ['metrics', 'value', 'category'])

for i in [df_coc, df_hra, df_total]:
    df_summary = pd.concat([df_summary, i], axis=0)
    
df_summary.sort_values(by=['metrics','category'],inplace=True)
display(df_summary)
df_summary.to_csv('final_blueprint_DR_types_CoCs.csv', index=False)

,metrics,value,category
0,preserved_units_chg,118830.0,CoCs
0,preserved_units_chg,162280.0,HRAs
0,preserved_units_chg,436062.0,total
1,subsidized_units_chg,39302.0,CoCs
1,subsidized_units_chg,56881.0,HRAs
1,subsidized_units_chg,120528.0,total
